Link : https://do.diba.cat/data/ds/establiments/graella?embed=true&filters=false&export-btn=false&iframe-btn=false&info=false&sort_field=descripcio_activitat&sort_ord=desc&start_item=6700

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
link_default = 'https://do.diba.cat/data/ds/establiments/graella?embed=true&filters=false&export-btn=false&iframe-btn=false&info=false&sort_field=descripcio_activitat&sort_ord=desc&start_item=0'

In [3]:
def extract_column_headings(link):
    html_text = requests.get(link_default).text
    soup = BeautifulSoup(html_text, 'lxml')
    table = soup.find('table', class_ = "table table-condensed table-striped table-hover text-nowrap bottom-scroll")
    
    # Save the column headings
    headings = []
    thead = soup.find('thead')
    heading_row = thead.find('tr')
    column_headings = heading_row.find_all('th')
    for column_heading in column_headings:
    #     print(column_heading)
        heading = column_heading.find('div', class_ = 'group-label')
        if heading:  # the first column is blank, so it doesn't contain div
    #         print(heading.text.strip())
            headings.append(heading.text.strip())
    # Add empty column heading at the start to match the data format in the table
    headings.insert(0, 'Empty')
    return headings

In [4]:
headings = extract_column_headings(link_default)
print(headings)

['Empty', 'Id', 'Nom comercial', 'Nom raó Social', 'NIF raó social', "Descripció de l'activitat", "Identificador de l'activitat", "Nom de l'ens", 'Codi INE', 'Ens relacionat', 'Codi de comarca', 'Adreça completa', 'UTMx', 'UTMy', 'Localització', 'Tipus de via', 'Nom de la via', 'Número via', 'Escala', 'Pis', 'Porta', 'Tipus de nucli', 'Codi postal', 'Municipi', 'Altres activitats', 'Sector econòmic', 'Codi NACE', 'Descripció NACE', 'Tipus Industrial o productiva', 'Codi CCAE', 'Descripció CCAE', 'Tipus comercial', 'Tipus EMA', 'Data de modificació', 'Codi Ens', 'Annex', 'Data llicència', 'Data CI', 'Conjunt de dades', 'Últim canvi']


In [5]:
def page_scraper(link, headings):
    html_text = requests.get(link).text
    soup = BeautifulSoup(html_text, 'lxml')
    table = soup.find('table', class_ = "table table-condensed table-striped table-hover text-nowrap bottom-scroll")
    tbody = soup.find('tbody')
    rows = tbody.find_all('tr')
    scraped_page = []
    for row_index, row in enumerate(rows):
        dict_row = {}
        cells = row.find_all('td')
        for index, cell in enumerate(cells):
            col_title = headings[index]
    #         print(cell.text.strip())
            dict_row[col_title] = cell.text.strip()
        scraped_page.append(dict_row) 
    return scraped_page

In [6]:
# 28000 in total
scraped_data = []
for i in range(0, 28000, 50):
    link = '='.join(link_default.split('=')[:-1]) + '=' + str(i)
#     print(link)
    scraped_data.extend(page_scraper(link, headings))

In [7]:
df = pd.DataFrame(scraped_data)
df

,Empty,Id,Nom comercial,Nom raó Social,NIF raó social,Descripció de l'activitat,Identificador de l'activitat,Nom de l'ens,Codi INE,Ens relacionat,...,Descripció CCAE,Tipus comercial,Tipus EMA,Data de modificació,Codi Ens,Annex,Data llicència,Data CI,Conjunt de dades,Últim canvi
0,,cens-0828450006-2602,AREA OPTICA SLL,AREA OPTICA SLL,B62997176,òptica,2602,Ajuntament de Tordera,08284,0828450006,...,,1,0,2019-06-25 00:00:00,0828450006,L18-Proj. i Cert.,2012-08-29 00:00:00,,Cens municipal d'activitats i establiments,2024-03-20 00:52
1,,cens-0828450006-2852,FEDEROPTICS TORDERA -ESPAI OPTIC LLORET,FEDEROPTICS TORDERA -ESPAI OPTIC LLORET,B17992629,òptica,2852,Ajuntament de Tordera,08284,0828450006,...,,1,0,2019-06-25 00:00:00,0828450006,L18-Cert.,2016-02-04 00:00:00,,Cens municipal d'activitats i establiments,2024-03-20 00:57
2,,cens-0827320002-668,ANTIGA ESCOLA DE CA L'AVI,AJUNTAMENT DE SUBIRATS,P0827300E,ÚS RELIGIÓS,668,Ajuntament de Subirats,08273,0827320002,...,,1,0,2022-02-22 00:00:00,0827320002,Sectorial,,,Cens municipal d'activitats i establiments,2024-03-20 01:02
3,,cens-0804780001-587,ÒPTICA JANÉ,,,Òptica i optometria,587,Ajuntament de Cardona,08047,0804780001,...,,1,0,2012-10-26 00:00:00,0804780001,L18-Cert.,2001-07-04 00:00:00,,Cens municipal d'activitats i establiments,2024-03-20 00:52
4,,cens-0811410007-2891,ÒPTICA UNIVERSITÀRIA,OPTICA DEL PENEDES S.L,B58442500,Òptica i audiologia,2891,Ajuntament de Martorell,08114,0811410007,...,Activitats de medicina especialitzada,1,0,2022-07-06 00:00:00,0811410007,L18-Proj. i Cert.,2022-07-05 00:00:00,2022-07-04 00:00:00,Cens municipal d'activitats i establiments,2024-03-20 01:03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27995,,cens-0822990004-54,CAMPING CAL PARADIS,,,Càmping i restaurant,54,Ajuntament de Sant Mateu de Bages,08229,0822990004,...,,1,0,2021-05-11 00:00:00,0822990004,II,2014-10-01 00:00:00,2014-02-17 00:00:00,Cens municipal d'activitats i establiments,2024-03-20 00:47
27996,,cens-0830360009-52,Càmping el Pont,,,Càmping,52,Ajuntament de Vilanova de Sau,08303,0830360009,...,,1,0,2023-02-01 00:00:00,0830360009,II,2002-12-24 00:00:00,2005-07-22 00:00:00,Cens municipal d'activitats i establiments,2024-03-20 00:47
27997,,cens-0806730008-182,Càmping Puigsagordi,Càmping Puigsagordi,B13921382,Càmping,182,Ajuntament de Centelles,08067,0806730008,...,Càmpings,1,0,2023-10-05 00:00:00,0806730008,II,2008-06-20 00:00:00,2003-02-25 00:00:00,Cens municipal d'activitats i establiments,2024-03-20 01:02
27998,,cens-0890190004-12,CAMPING RUPIT,"Càmping Glòria, SA",B58805078,Càmping,12,Ajuntament de Rupit i Pruit,08901,0890190004,...,,1,0,2021-11-16 00:00:00,0890190004,II,2007-07-04 00:00:00,,Cens municipal d'activitats i establiments,2024-03-20 01:03


In [8]:
df.to_csv('establishments_Catalonia.csv', index=False)

In [9]:
df.head()

,Empty,Id,Nom comercial,Nom raó Social,NIF raó social,Descripció de l'activitat,Identificador de l'activitat,Nom de l'ens,Codi INE,Ens relacionat,...,Descripció CCAE,Tipus comercial,Tipus EMA,Data de modificació,Codi Ens,Annex,Data llicència,Data CI,Conjunt de dades,Últim canvi
0,,cens-0828450006-2602,AREA OPTICA SLL,AREA OPTICA SLL,B62997176,òptica,2602,Ajuntament de Tordera,08284,0828450006,...,,1,0,2019-06-25 00:00:00,0828450006,L18-Proj. i Cert.,2012-08-29 00:00:00,,Cens municipal d'activitats i establiments,2024-03-20 00:52
1,,cens-0828450006-2852,FEDEROPTICS TORDERA -ESPAI OPTIC LLORET,FEDEROPTICS TORDERA -ESPAI OPTIC LLORET,B17992629,òptica,2852,Ajuntament de Tordera,08284,0828450006,...,,1,0,2019-06-25 00:00:00,0828450006,L18-Cert.,2016-02-04 00:00:00,,Cens municipal d'activitats i establiments,2024-03-20 00:57
2,,cens-0827320002-668,ANTIGA ESCOLA DE CA L'AVI,AJUNTAMENT DE SUBIRATS,P0827300E,ÚS RELIGIÓS,668,Ajuntament de Subirats,08273,0827320002,...,,1,0,2022-02-22 00:00:00,0827320002,Sectorial,,,Cens municipal d'activitats i establiments,2024-03-20 01:02
3,,cens-0804780001-587,ÒPTICA JANÉ,,,Òptica i optometria,587,Ajuntament de Cardona,08047,0804780001,...,,1,0,2012-10-26 00:00:00,0804780001,L18-Cert.,2001-07-04 00:00:00,,Cens municipal d'activitats i establiments,2024-03-20 00:52
4,,cens-0811410007-2891,ÒPTICA UNIVERSITÀRIA,OPTICA DEL PENEDES S.L,B58442500,Òptica i audiologia,2891,Ajuntament de Martorell,08114,0811410007,...,Activitats de medicina especialitzada,1,0,2022-07-06 00:00:00,0811410007,L18-Proj. i Cert.,2022-07-05 00:00:00,2022-07-04 00:00:00,Cens municipal d'activitats i establiments,2024-03-20 01:03
